In [ ]:
#@title Model

model = '2.2-diffusers' #@param ["2.0", "2.1", "2.1-diffusers", "2.2-diffusers"]

In [ ]:
#@title Settings

branch = 'main' #@param ["main", "dev"]
use_ngrok = False #@param {type:"boolean"}
ngrok_auth_key = "" #@param {type:"string"}
use_localtunnel = False #@param {type:"boolean"}
use_cloudflared = False #@param {type:"boolean"}
use_flash_attention = False #@param {type:"boolean"}
save_models_to_gdrive = False #@param {type:"boolean"}
save_images_to_gdrive = False #@param {type:"boolean"}

use_diffusers = False

if model == "2.0":
  model_name = "kd20"

if model == "2.1":
  model_name = "kd21"

if model == "2.1-diffusers":
  model_name = "kd21"
  use_diffusers = True

if model == "2.2-diffusers":
  model_name = "kd22"
  use_diffusers = True


In [ ]:
#@title Diffusers
optimizations = []

half_weights = True #@param {type:"boolean"}
use_xformers = False #@param {type:"boolean"}
use_sliced_attention = False #@param {type:"boolean"}
use_sequential_offload = False #@param {type:"boolean"}
channels_last_memory = False #@param {type:"boolean"}

if half_weights: optimizations.append('half_weights')
if use_xformers: optimizations.append('xformers')
if use_sliced_attention: optimizations.append('sliced_attention')
if use_sequential_offload: optimizations.append('sequential_offload')
if channels_last_memory: optimizations.append('channels_last')

optimizations = str.join(',', optimizations)

In [ ]:
#@title Extensions
disabled_extensions = []

image_browser = True #@param {type:"boolean"}
interrogator = False #@param {type:"boolean"}
mesh_gen = False #@param {type:"boolean"}
prompt_styles = True #@param {type:"boolean"}
segmentation = False #@param {type:"boolean"}
upscaler = True #@param {type:"boolean"}
training = False #@param {type:"boolean"}

if not image_browser: disabled_extensions.append('kd-image-browser')
if not interrogator: disabled_extensions.append('kd-interrogator')
if not mesh_gen: disabled_extensions.append('kd-mesh-gen')
if not prompt_styles: disabled_extensions.append('kd-prompt-styles')
if not segmentation: disabled_extensions.append('kd-segmentation')
if not upscaler: disabled_extensions.append('kd-upscaler')
if not training: disabled_extensions.append('kd-training')

disabled_extensions = str.join(',', disabled_extensions)

In [ ]:
#@title UI

side_tabs_view = True #@param {type:"boolean"}

In [ ]:
if save_models_to_gdrive or save_images_to_gdrive:
  from google.colab import drive

  drive.mount('/content/gdrive')

In [ ]:
!python --version

[gpu] = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print (gpu)

import torch
print(torch.__version__)

!nvidia-smi

In [ ]:
%cd /content

!git clone -b {branch} https://github.com/seruva19/kubin.git
%cd /content/kubin

!pip install -r requirements.txt

if use_diffusers:
  !pip install -r diffusers/requirements.txt

if use_flash_attention:
  !pip install https://github.com/seruva19/flash-attn-wheels/raw/main/torch2.0.0%2Bcu118/flash_attn-1.0.1-cp310-cp310-linux_x86_64.whl

if use_xformers:
  !pip install xformers==0.0.20
  !python -m xformers.info

In [ ]:
if use_ngrok:
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_auth_key)

  tunnels = ngrok.get_tunnels()
  for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

  print(ngrok.connect(7860, "http"))

In [ ]:
if use_localtunnel:
    !npm install -g localtunnel
    lturl = !curl ipv4.icanhazip.com
    print("endpoint IP for localtunnel:")
    print(lturl[0])

In [ ]:
if use_cloudflared:
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb

In [ ]:
!kill -9 $(lsof -t -i tcp:7860)

!python src/kubin.py \
  --model-name={model_name} \
  --cache-dir={"/content/gdrive/MyDrive/kubin/models" if save_models_to_gdrive else "/content/kubin/models"} \
  --output-dir={"/content/gdrive/MyDrive/kubin/output" if save_images_to_gdrive else "/content/kubin/output"} \
  --disabled-extensions={disabled_extensions} \
  --pipeline={"diffusers" if use_diffusers else "native"} \
  --optimize={optimizations} \
  {"--side-tabs='use'" if side_tabs_view else ""} \
  --share={"none" if use_ngrok or use_localtunnel or use_cloudflared else "gradio"} \
  {"--flash-attention='use'" if use_flash_attention else ":"} & \
  {"npx localtunnel --port 7860" if use_localtunnel else ":"} & \
  {"cloudflared tunnel --url http://localhost:7860" if use_cloudflared else ":"}